In [ ]:
pip install pandas

In [ ]:
pip install line-bot-sdk


In [ ]:
from flask import Flask, request, abort
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)
import pandas as pd
import os

# 從環境變數中獲取 Token 和 Secret
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')

# 初始化 Flask
app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)
# 台灣所有縣市的代碼
cities = {
    "台北": "6300100",
    "新北": "6500100",
    "基隆": "1001701",
    "桃園": "6800100",
    "新竹市": "1001801",
    "新竹縣": "1000401",
    "苗栗": "1000501",
    "台中": "6600100",
    "彰化": "1000701",
    "南投": "1000801",
    "雲林": "1000901",
    "嘉義市": "1002001",
    "嘉義縣": "1001001",
    "台南": "6700100",
    "高雄": "6400100",
    "屏東": "1001301",
    "宜蘭": "1000201",
    "花蓮": "1001501",
    "台東": "1001401",
    "澎湖": "1001601",
    "金門": "0902001",
    "連江": "0900701"
}

# 讀取旅館資料
hotel_data = pd.read_csv('./Hotel_E_F.csv')

# 使用者選擇的城市和地區紀錄
user_city = {}

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    except Exception as e:
        print(f"Error: {e}")
        abort(400)
        
    return 'OK'

# 處理文字訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text.strip()
    user_id = event.source.user_id

    # 每次事件都初始化 API Client
    with ApiClient(Configuration(access_token=line_access_token)) as api_client:
        line_bot_api = MessagingApi(api_client)
        
        if user_message == "選擇城市":
            reply_text = (
                "請選擇想去的縣市：\n台北、新北、基隆、桃園、新竹市、新竹縣、苗栗、台中、彰化、南投、雲林、嘉義市、嘉義縣、台南、高雄、屏東、宜蘭、花蓮、台東、澎湖、金門、連江\n"
                "格式：<城市> <地區>\n例如：台北 士林 或 台中 西區"
            )
        elif " " in user_message:
            city, district = user_message.split(" ", 1)
            if city in cities:
                user_city[user_id] = {"city": city, "district": district}
                reply_text = f"您已選擇城市：{city}，地區：{district}\n接下來可以輸入「即時資訊」或「住宿資訊」來查詢資料。"
            else:
                reply_text = "無效的城市名稱，請重新輸入。"
        elif user_message == "即時資訊":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_code = cities[city_info['city']]
                weather_url = f"https://www.cwb.gov.tw/V8/C/W/Town/Town.html?TID={city_code}"
                traffic_info = f"https://maps.google.com/?q={city_info['city']} {city_info['district']} 交通"
                reply_text = (
                    f"{city_info['city']} {city_info['district']} 的天氣：\n{weather_url}\n\n"
                    f"即時路況：\n{traffic_info}"
                )
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「即時資訊」。"
        elif user_message == "住宿資訊":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_hotels = hotel_data[
                    (hotel_data['城市'] == city_info['city']) | 
                    (hotel_data['地址'].str.contains(city_info['district']))
                ]
                if not city_hotels.empty:
                    reply_text = "【住宿資訊】\n"
                    for index, row in city_hotels.iterrows():
                        reply_text += f"🏨 {row['旅館名稱']}\n📞 {row['電話']}\n📍 {row['地址']}\n💰 價格：{row['最低房價']} ~ {row['最高房價']}\n---\n"
                else:
                    reply_text = f"找不到 {city_info['city']} {city_info['district']} 的住宿資訊。"
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「住宿資訊」。"
        else:
            reply_text = "請輸入正確的指令或輸入「選擇城市」查看支援的城市。"

        # 回傳訊息
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/May/2025 12:37:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 12:37:36] "POST / HTTP/1.1" 200 -
